In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
import cmath #for complex numbers
from scipy.integrate import quad #for numerical integration
from sklearn.preprocessing import MinMaxScaler
import scipy
import time
import multiprocessing

In [125]:
num_model_parameters = 6
num_strikes = 8
num_maturities = 8


num_input_parameters = 6
num_output_parameters = num_maturities*num_strikes
learning_rate = 0.0001
num_steps = 5
batch_size = 10
num_neurons = 40

#initial values
S0 = 1.0
V0 = 0.05
r = 0.0


contract_bounds = np.array([[0.8*S0,1.2*S0],[5,10]]) #bounds for K,T
model_bounds = np.array([[1,2],[0.2,0.8],[-1,0],[2,5],[0.05,0.1],[0.1,0.3]]) #bounds for alpha,beta,rho,a,b,c, make sure alpha>0,2ab>c^2

"""
Note: The grid of stirkes and maturities is equidistant here put could be choosen differently for real world application.
Note: For the code below to striktly follow the bounds specified above make sure that *_distance x num_* is less than half the distance from the highest to lowest * (* = strikes/maturities). 
"""
maturities_distance = (contract_bounds[1,1]-contract_bounds[1,0])/(2*num_maturities) 
strikes_distance = (contract_bounds[0,1]-contract_bounds[0,0])/(2*num_strikes)

strikes = np.linspace(contract_bounds[0,0],contract_bounds[0,0]+num_strikes*strikes_distance,num_strikes)
maturities = np.linspace(contract_bounds[1,0],contract_bounds[1,0]+num_maturities*maturities_distance,num_maturities)

In [126]:
def corr_brownian_motion(n, T, dim, rho):
    dt = T/n

    dW1 = norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
    dW2 = rho * dW1 + np.sqrt(1 - np.power(rho ,2)) * norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
        
    W1 = np.cumsum(dW1, axis=-1)
    W2 = np.cumsum(dW2, axis=-1)
 
    return W1,W2

def euler_maruyama(mu,sigma,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + np.multiply(mu(Y[l,i],l,i),dt) + sigma(Y[l,i],l,i)*sqrt_dt*(W[l,i+1]-W[l,i])
    
    return Y

def heston_SLV(alpha,beta,a,b,c,T,W,Z,V0,S0):
   
    assert(2*a*b > c*c)

    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))
    
    V = euler_maruyama(mu2,sigma2,T,V0,Z)
    
    def mu1(S,i,k):
        return np.multiply(r,S)
    
    def sigma1(S,i,k):
        return alpha*np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),np.power(S,1+beta))
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def reverse_transform_X(X_scaled):
    X = np.zeros(X_scaled.shape)
    for i in range(num_input_parameters):
        X[:,i] = X_scaled[:,i]*(model_bounds[i][1]-model_bounds[i][0]) + model_bounds[i][0]
    return X


def implied_vol(P,K,T):

    if not P<S0:
        return -2
    if not P>=S0-K*np.exp(-r*T):
        return -1
        #P = S0-K*np.exp(-r*T) + 0.001

    def f(sigma):
        dplus = (np.log(S0 / K) + (r  + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        dminus = (np.log(S0 / K) + (r  - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        
        return S0 * norm.cdf(dplus, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(dminus, 0.0, 1.0) - P
     
    return scipy.optimize.brentq(f, 0, 1)

def BS_call_price(sigma,K,T):
    dplus = (np.log(S0 / K) + (r  + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    dminus = (np.log(S0 / K) + (r  - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    return S0 * norm.cdf(dplus, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(dminus, 0.0, 1.0)

def HeMC(rho, kappa, theta, xi, T, n):

    # Generate a Monte Carlo simulation for the Heston model

    # Generate random Brownian Motion
    mu = 0
    dt = T/n
    MU  = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W_   = np.random.multivariate_normal(MU, COV, n)
    W_S = W_[:,0]
    W_v = W_[:,1]

    # Generate paths
    vt    = np.zeros(n+1)
    vt[0] = V0
    St    = np.zeros(n+1)
    St[0] = S0
    for t in range(1,n):
        vt[t] = np.abs(vt[t-1] + kappa*(theta-np.abs(vt[t-1]))*dt + xi*np.sqrt(np.abs(vt[t-1])*dt)*W_v[t])
        St[t] = St[t-1]*np.exp((mu - 0.5*vt[t-1])*dt + np.sqrt(vt[t-1]*dt)*W_S[t])

    return St

def heston_closed(a,b,c,T,K,rho,V0,S0,r=0):
    
    def char_f(w,a,b,c,T,K,rho,V0,S0,r):
        alpha = -0.5*w*(w+complex(0,1))
        beta = a - rho*c*w*complex(0,1)
        gamma = c*c/2
        h = cmath.sqrt(beta*beta-4*alpha*gamma)
        rplus = (beta + h)/c/c
        rminus = (beta - h)/c/c
        g = rminus/rplus
        D =  rminus*(1-cmath.exp(-h*T))/(1-g*cmath.exp(-h*T))
        C = a*(rminus*T-2/c/c*cmath.log((1-g*cmath.exp(-h*T))/(1-g)))
        return cmath.exp(C*b+D*V0+complex(0,1)*w*cmath.log(S0*cmath.exp(r*T)))

    def integrand1(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w-i,a,b,c,T,K,rho,V0,S0,r)/i/w/char_f(-i,a,b,c,T,K,rho,V0,S0,r)).real
    
    def integrand2(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w,a,b,c,T,K,rho,V0,S0,r)/i/w).real
    
    pi1 = 0.5 + quad(integrand1,0,np.inf)[0]/np.pi
    pi2 = 0.5 + quad(integrand2,0,np.inf)[0]/np.pi
    
    return (S0*pi1 + cmath.exp(-r*T)*K*pi2).real

def next_batch_sabr_EM_train(batch_size,contract_bounds,model_bounds):
    X_scaled = np.zeros((batch_size,num_input_parameters))
    y = np.zeros((batch_size,num_output_parameters))

    X_scaled[:,0] = uniform.rvs(size=batch_size) #alpha
    X_scaled[:,1] = uniform.rvs(size=batch_size) #beta
    X_scaled[:,2] = uniform.rvs(size=batch_size) #rho
    X_scaled[:,3] = uniform.rvs(size=batch_size) #a
    X_scaled[:,4] = uniform.rvs(size=batch_size) #b
    X_scaled[:,5] = uniform.rvs(size=batch_size) #c

    X = reverse_transform_X(X_scaled)
    #test
    #X[:,0] = np.ones(batch_size)
    #X[:,1] = np.zeros(batch_size)

    n = 100
    dim = 10000
    for i in range(batch_size):
        W,Z = corr_brownian_motion(n,maturities[-1],dim,X[i,2])
        S,V = heston_SLV(X[i,0],X[i,1],X[i,3],X[i,4],X[i,5],maturities[-1],W,Z,V0,S0)
        #S2 = np.zeros((dim,n+1))
        #for c in range(dim):
        #    S2[c,:] = HeMC(X[i,2],X[i,3],X[i,4],X[i,5],maturities[-1],n)
        
        for j in range(num_maturities):
            n_current = int(maturities[j]/maturities[-1]*n)
            S_T = S[:,n_current]
            #S_T2 = S2[:,n_current]
            
            for k in range(num_strikes):
                P = np.mean(np.maximum(S_T-np.ones(dim)*strikes[k],np.zeros(dim)))*np.exp(-r*maturities[j])
                #P2 = np.mean(np.maximum(S_T2-np.ones(dim)*strikes[k],np.zeros(dim)))*np.exp(-r*maturities[j])
                #P3 = heston_closed(X[i,3],X[i,4],X[i,5],maturities[j],strikes[k],X[i,2],V0,S0,r=0)
                y[i,j*num_strikes+k] = implied_vol(P,strikes[k],maturities[j])
                #print(P,P2,P3,strikes[k],maturities[j])
                #y[i,j*num_strikes+k] = np.exp(-r*maturities[j])*np.mean(np.maximum(S_T-np.ones(dim)*strikes[k],np.zeros(dim)))
    return X_scaled,y

In [128]:
next_batch_sabr_EM_train(1,contract_bounds,model_bounds)

(array([[0.30301405, 0.77021207, 0.85837102, 0.06397085, 0.74028526,
         0.56952407]]),
 array([[0.09448701, 0.09602261, 0.09748525, 0.09878144, 0.10009577,
         0.1012544 , 0.10239714, 0.        , 0.09486004, 0.09634059,
         0.09760564, 0.0988013 , 0.09992522, 0.10107924, 0.10217908,
         0.        , 0.0972881 , 0.09826108, 0.0993131 , 0.10030908,
         0.10133894, 0.1023775 , 0.10332805, 0.        , 0.09511892,
         0.09632522, 0.0976522 , 0.09885138, 0.09990906, 0.10092588,
         0.10197475, 0.        , 0.09494141, 0.09626996, 0.09754379,
         0.09874432, 0.0998952 , 0.10096026, 0.10200676, 0.        ,
         0.09625029, 0.09735342, 0.09840566, 0.09954066, 0.10060573,
         0.10165873, 0.10267205, 0.        , 0.0962775 , 0.09745232,
         0.09859472, 0.09963014, 0.10067738, 0.10164949, 0.1027085 ,
         0.        , 0.09909198, 0.09993383, 0.10084462, 0.1018381 ,
         0.10285886, 0.103893  , 0.10483671, 0.        ]]))

-1.0